<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/notebook_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Das Projekt aus dem Github klonen und in den Projektsordner**

In [5]:
#wenn die Ordner noch nicht geklont ist, soll dieser Fehler zuerst durchgeführt werden.
!git clone https://github.com/hanhluukim/replication-topic-modelling-in-embedding-space.git

Cloning into 'replication-topic-modelling-in-embedding-space'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 287 (delta 39), reused 41 (delta 20), pack-reused 225
Receiving objects: 100% (287/287), 4.52 MiB | 7.04 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [6]:
cd /content/replication-topic-modelling-in-embedding-space

/content/replication-topic-modelling-in-embedding-space


#**Die benötige Paketen für das Projekt mittels requirements.txt installieren**



In [7]:
# Falls die Packages noch nicht installiert wurden, 
!pip install -r "/content/replication-topic-modelling-in-embedding-space/requirements.txt"

     |████████████████████████████████| 4.0 MB 4.5 MB/s 
     |████████████████████████████████| 88 kB 6.1 MB/s 
     |████████████████████████████████| 28.8 MB 169 kB/s 
     |████████████████████████████████| 636 kB 38.9 MB/s 
     |████████████████████████████████| 79.9 MB 56 kB/s 
     |████████████████████████████████| 880 kB 37.8 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
     |████████████████████████████████| 1.1 MB 37.3 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=e222cd2de6e5be9161e1d8918e26a798b5a26e816640031f9f05bc8e9b7cae1d
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=f307610a40b5b18c4b9fa87827a505f936cc82a9b1274559e348f7b9e336667f
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80

# **Gebrauchte Paketen importieren**

In [8]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import umap.umap_ as umap
import time
import plotly.express as px
from sklearn import cluster
from sklearn import metrics

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Vorverarbeitung und BOW-Repräsentationen für Textdaten durchführen**
1. Vocabular erstellen
2. BOW-Repräsentationen für allen Teildatensätzen

In [9]:
from src.preprare_dataset import TextDataLoader

In [10]:
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")
# Beispiel von Textdaten
textsloader.show_example_raw_texts(n_docs=2)

loading texts: ...
finished load!
check some sample texts of the dataset
['From', ':', 'lerxst', '@', 'wam', '.', 'umd', '.', 'edu', '(', "where's", 'my', 'thing', ')', 'Subject', ':', 'WHAT', 'car', 'is', 'this', '!', '?', 'Nntp', 'Posting', 'Host', ':', 'rac3', '.', 'wam', '.', 'umd', '.', 'edu', 'Organization', ':', 'University', 'of', 'Maryland', ',', 'College', 'Park', 'Lines', ':', '15', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2', 'door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s', '/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs'

In [11]:
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)

start: preprocessing: ...
finised: preprocessing!


In [12]:
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=10, stopwords_remove_from_voca=True)

permuted indices for the train set: [187 125 213 147 235  17  85  93 199 223 146 127 180  43 243]
start creating vocabulary ...
length of the vocabulary: 348
sample ten words of the vocabulary: ['full', 'type', 'good', 'sense', 'true', 'clear', 'program', 'folks', 'project', 'car']
length word2id list: 348
length id2word list: 348
finished: creating vocabulary


In [13]:
# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False 
word2id, id2word, train_set, test_set, val_set = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)

length train-documents-indices : 4654
length of the vocabulary: 348


start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 149
min doc-id: 0
all docs: 4654
all words: 4654
docidx unique 150
words unique: 348
ndocs: 150
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 1465
all words: 1465
docidx unique 50
words unique: 325
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [1, 2, 4, 6, 7, 8, 9, 10, 11, 12]
max word-id: 346
min word-id: 1
max doc-id: 49
min doc-id: 0
all docs: 719
all words: 719
docidx unique 50
words unique: 240
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 6,

# **Vocabular und IDs anzeigen als Beispiel**

---





In [14]:
# show for samples: 100 word2id and id2 word
word2id_df_100 = pd.DataFrame()
word2id_df_100['word'] = list(word2id.keys())[:100]
word2id_df_100['id'] = list(word2id.values())[:100]
word2id_df_100

,word,id
0,full,0
1,type,1
2,good,2
3,sense,3
4,true,4
...,...,...
95,things,95
96,sound,96
97,center,97
98,response,98


# **Die Größe von Datensätzen kontrollieren**

In [15]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train_set["tokens"])}')
print(f'Size of val set: {len(val_set["tokens"])}')
print(f'Size of test set: {len(test_set["test"]["tokens"])}')

Size of the vocabulary after prprocessing ist: 348
Size of train set: 150
Size of val set: 100
Size of test set: 50


# **Dokumenten wiederstellen für Word2Vec Embedding**

---



In [16]:
# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

,text-after-preprocessing
0,article institute nntp posting host article wr...
1,nntp posting host newsreader tin version de wr...
2,nntp posting host disclaimer expressed disclai...
3,article ca ca writes east idea rest world wash...
4,file university engineering computer network c...
...,...
95,university california article apr writes artic...
96,james space news nntp posting host reply artic...
97,cc nntp posting host cc reply cc university ar...
98,jim article reply distribution usa computer nn...


# **Word-Embedding trainieren mit dem Traindatensatz**

In [17]:
from src.embedding import WordEmbeddingCreator
save_path = Path.joinpath(Path.cwd(), "vocab_embedding.txt")
wb_creator = WordEmbeddingCreator(model_name="cbow", documents = docs_tr, save_path= save_path)
wb_creator.train(min_count=0, embedding_size= 10)
vocab = list(word2id.keys())
wb_creator.create_and_save_vocab_embedding(vocab, save_path)

word-embedding train begins
word-embedding train finished
length of vocabulary from word-embedding model 348
length of the vocabulary of prepraring-dataset-vocabulary: 348


100%|██████████| 348/348 [00:00<00:00, 16845.57it/s]


True

In [18]:
v = list(wb_creator.model.wv.vocab)[0]
vec = list(wb_creator.model.wv.__getitem__(v))
print(f'word-embedding of the word-- {v}: ')
print(f'vector: {vec}')
print(f'dim of vector: {len(vec)}')

word-embedding of the word-- article: 
vector: [-0.020197343, 0.041284982, -0.053527184, 0.14206944, 0.12212627, 0.10857858, 0.08356705, -0.06387062, 0.22258864, 0.3019734]
dim of vector: 10


# **Word-Embeddings visualieren als Beispiel**


In [19]:
# read word-embedding files
with open(save_path) as f:
  lines = f.readlines()
embedding_data = []
words_data = []
for t in lines:
  w = t.split("\t")[0]
  v = [float(e) for e in t.split("\t")[1].split(" ")]
  words_data.append(w)
  embedding_data.append(v)

In [20]:
# clustering words with KMeans and Words-Vectors
kmeans = cluster.KMeans(n_clusters=10)
kmeans.fit(embedding_data)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)

Cluster id labels for inputted data
[5 3 5 5 5 1 0 9 8 6 7 4 4 7 4 6 5 0 7 9 7 7 4 9 9 5 2 1 1 3 9 8 5 5 5 4 1
 4 9 6 8 7 8 4 3 7 5 9 7 4 5 7 9 4 8 7 6 0 6 7 4 1 0 8 6 3 0 2 4 8 4 8 9 4
 0 2 0 1 4 9 6 7 4 4 8 0 3 4 0 4 7 0 4 6 5 9 4 0 8 7 7 7 8 6 5 4 0 7 0 7 0
 8 0 4 9 7 7 6 6 7 1 0 9 2 3 8 4 4 3 3 8 8 6 0 1 7 1 9 6 4 7 4 3 3 9 1 1 3
 8 0 8 7 9 8 2 6 1 1 9 3 6 9 9 7 4 0 6 1 9 6 1 6 6 7 4 9 0 0 4 4 9 3 3 0 1
 0 2 9 9 9 9 1 3 8 1 3 8 7 2 3 6 2 3 6 3 7 7 9 1 3 6 9 1 0 4 3 7 0 3 5 4 3
 2 1 2 2 7 1 2 1 2 9 4 2 3 8 1 9 7 1 6 0 3 9 3 9 3 3 2 9 4 3 3 6 9 0 9 0 8
 6 5 4 9 7 6 4 6 7 9 3 3 8 6 4 4 4 6 3 1 3 4 0 0 0 8 6 2 1 8 2 2 0 0 3 9 8
 0 4 6 8 8 8 1 2 3 8 3 9 6 7 9 3 7 3 0 2 9 2 0 0 8 0 6 2 8 3 6 7 0 8 3 2 8
 3 3 0 6 0 7 0 2 7 0 2 6 9 3 3]
Centroids data
[[-0.00939792  0.01633509 -0.00320749  0.0543278   0.07340219  0.04369927
   0.03705035 -0.04072507  0.06410781  0.09952773]
 [-0.05341788  0.0288578  -0.08216598  0.09082107  0.09640938  0.05582194
   0.04887861 -0.06703641  0.14821184  0.18

In [21]:
# dimension reduction with umap
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(embedding_data)
print('Duration: {} seconds'.format(time.time() - start))

# show samples after dim-reduction in dataframe
wb = pd.DataFrame(embedding, columns=['x', 'y', 'z'])
wb['word'] = words_data
wb['cluster'] = ['cluster ' + str(c) for c in labels]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Duration: 18.07043766975403 seconds


In [22]:
# visualization
fig = px.scatter_3d(wb, 
                    text = wb['word'],
                    x='x', y='y', z='z',
                    color = wb['cluster'],
                    title ="word-embedding-samples")
fig.show()

# **ETM-Model trainieren**

In [23]:
# using DocSet to use easier the modul DataSet from torch
from src.train_etm import DocSet, ETMTrain
from src.etm import ETM

vocab_size = len(list(word2id.keys()))
tr_set = DocSet("train", vocab_size, train_set)
print(len(tr_set))
print(tr_set.__getitem__(0))

150
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 2., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 

In [24]:
num_topics = 5
t_hidden_size = 100
rho_size = len(embedding_data[0])
emb_size = len(embedding_data[0])
theta_act = "relu"

class TrainArguments:
      def __init__(self, epochs, batch_size, log_interval):
          self.epochs = epochs
          self.batch_size = batch_size
          self.log_interval = log_interval

class OptimizerArguments:
      def __init__(self, optimizer_name, lr, wdecay):
            self.optimizer = optimizer_name
            self.lr = lr
            self.wdecay = wdecay
            
train_args = TrainArguments(epochs=10, batch_size=100, log_interval=None)
optimizer_args = OptimizerArguments(optimizer_name="Adam", lr=0.005, wdecay=0.1)

print(train_args.epochs)
print(optimizer_args.optimizer)

training_set = train_set

# define the ETM-model with setting-parameters
etm_model = ETM(
      num_topics, 
      vocab_size, 
      t_hidden_size, rho_size, emb_size, theta_act, 
      embedding_data, enc_drop=0.5)

# start training
train_class = ETMTrain().train(
    etm_model,
    vocab_size, 
    train_args, optimizer_args, training_set, 
    num_topics, t_hidden_size, rho_size, emb_size, theta_act, embedding_data, 0.5)

10
Adam
Defaulting to vanilla SGD
Epoch 0/10:
torch.Size([100])
torch.Size([])
total loss: 17575.361328125
Epoch 1/10:
torch.Size([100])
torch.Size([])
total loss: 17677.140625
Epoch 2/10:
torch.Size([100])
torch.Size([])
total loss: 17759.125
Epoch 3/10:
torch.Size([100])
torch.Size([])
total loss: 17727.845703125
Epoch 4/10:
torch.Size([100])
torch.Size([])
total loss: 19209.767578125
Epoch 5/10:
torch.Size([100])
torch.Size([])
total loss: 17325.36328125
Epoch 6/10:
torch.Size([100])
torch.Size([])
total loss: 17405.5234375
Epoch 7/10:
torch.Size([100])
torch.Size([])
total loss: 18358.8203125
Epoch 8/10:
torch.Size([100])
torch.Size([])
total loss: 17405.419921875
Epoch 9/10:
torch.Size([100])
torch.Size([])
total loss: 18225.072265625
